In [1]:
import numpy as np
from neuropy import ProcessData
basepath =  '/data/Clustering/SleepDeprivation/RatN_Day1_test_neuropy'
sess = ProcessData(basepath=basepath)

## Check recording info

In [4]:
print(sess.recinfo)

filename: /data/Clustering/SleepDeprivation/RatN_Day1_test_neuropy/RatN_Day1_2019-10-09_03-52-32.xml 
# channels: 134
sampling rate: 30000
lfp Srate (downsampled): 1250


## Set probe configuration
Lets create probe configuration for a 128-channel probe

In [ ]:
from neuropy.core import Shank, Probe
from neuropy.plotting import plot_probe
shanks = []
channel_groups = sess.recinfo.channel_groups
for i in range(8):

    shank = Shank(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=20,
        y_shift_per_column=[0,-7.5],
        channel_id=np.append(channel_groups[i][::2][::-1],channel_groups[i][1::2][::-1]),
    )
    shanks.append(shank)
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)

probe1 = Probe(shanks)
sess.probegroup.add_probe(probe1)

# probe2 = Probe(shanks)
# probe2.move((probe1.x_max+400,0))
# sess.probegroup.add_probe(probe2)
sess.probegroup.save()

plot_probe(sess.probegroup)




## Create epochs for your experimental paradigm
A typical experiment involves multiple epochs such pre sleep, running on track and then another sleep epoch.

In [ ]:
import pandas as pd

epochs = pd.DataFrame(
    {
        "start": [0,10000,20000],
        "stop": [9000,13600,30000],
        "label": ["pre", "maze", "post"],
    }
)
sess.paradigm.add_epochs(epochs)
sess.paradigm.epochs
# sess.paradigm.save() #Uncomment to save file


# Detect epochs
Here we will various types of epochs which typical for hippocampal recordings.

In [9]:
from neuropy import analyses,plotting

## Artifacts epochs
A typical session will have some artifacts that may negatively influence many analyses. Using a simple zscore measure, we can identify epochs where signal is above some threshold.

In [55]:
signal = sess.eegfile.get_signal([1])
artifact_epochs = analyses.detect_artifact_epochs(signal,thresh=7)
artifact_epochs.filename = sess.filePrefix.with_suffix('.artifact.npy')
artifact_epochs.save()
plotting.plot_artifact_epochs(artifact_epochs,signal)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='Time (s)', ylabel='Absolute zscore'>

## Ripple epochs
To detect ripples one also needs probegroup.

In [ ]:
signal = sess.eegfile.get_signal()
ripple_epochs = analyses.detect_ripple_epochs(signal,sess.probegroup)
# ripple_epochs.filename = sess.filePrefix.with_suffix('.ripple.npy')
# ripple_epochs.save()
plotting

In [10]:
signal = sess.eegfile.get_signal(channel_id=[1,2,3,4],t_start=1,t_stop=1.2)
plotting.plot_signal_traces(signal)

AttributeError: module 'ModulesPath.plotting' has no attribute 'plot_signal_traces'

In [ ]:
sess.ripple.metadata['params']